In [35]:
import numpy as np
import pandas as pd
from os import getcwd
from os.path import join, dirname
import string_utils
%matplotlib inline

DATA_DIR = join(dirname(getcwd()), 'data')
DATA_FILE = join(DATA_DIR, 'WebsiteUploads_dbo_ProductInventory_010.csv')
headers = ['sku', 'bulk_id', 'size_code', 'quantity_on_hand', 'lifecycle_status_flag', 'expected_date',
           'location', 'back_order_type', 'product_code', 'product_type', 'is_perishable', 'is_dropship',
           'street_address_required', 'effective_on', 'expired_on', 'added_on', 'added_by', 'modified_on',
           'modified_by', 'version']
column_types = {
    'sku': str, 
    'bulk_id': str, 
    'size_code': str,
    'is_perishable': bool,
    'is_dropship': bool,
    'street_address_required': bool
}
df = pd.read_csv(DATA_FILE, names=headers, parse_dates=[5, 13, 14, 15, 17], dtype=column_types)

OUT_OF_STOCK_FLAGS = ['DS', 'ED']
BACKORDERED_FLAGS = ['DX', 'EX']
STOCK_STATUS_LABELS = {
    0: 'out_of_stock',
    1: 'in_stock',
    2: 'back_ordered'
}

# assign in_stock, out_of_stock, back_ordered labels to each row
# assign date to row using effective on
# determine sku "stock_quotient" for a given day by determining if the product spent most of the day in stock

In [39]:
# todo: review this logic
def assign_stock_status(row):
    """determine the stock status of a sku"""
    if row['lifecycle_status_flag'] in OUT_OF_STOCK_FLAGS:
        return 0
    elif row['expected_date'] and row['lifecycle_status_flag'] in BACKORDERED_FLAGS:
        return 2
    else:
        return 1

df['effective_on_date'] = df['effective_on'].apply(lambda x: x.date())
df['stock_status'] = df.apply(assign_stock_status, axis=1)

In [103]:
import datetime

df = pd.DataFrame(data=[['a', datetime.datetime(2017, 10, 1, 5), datetime.datetime(2017, 10, 2, 5), 1],
                        ['a', datetime.datetime(2017, 10, 2, 5), datetime.datetime(2017, 10, 2, 13), 1],
                        ['a', datetime.datetime(2017, 10, 2, 13), datetime.datetime(2017, 10, 2, 14), -1],
                        ['a', datetime.datetime(2017, 10, 2, 14), datetime.datetime(2017, 10, 5, 14), 1]],
                  index=[1, 2, 3, 4],
                  columns=['grp', 'date', 'end_date', 'flag'])
df['dt'] = df['date'].apply(lambda x: x.date())
df['end_dt'] = df['end_date'].apply(lambda x: x.date())
df['internal_duration'] = df.apply(lambda r: (r['dt'] + datetime.timedelta(days=1)) - r['date'], axis=1)

TypeError: ("descriptor '__sub__' requires a 'datetime.datetime' object but received a 'datetime.date'", 'occurred at index 1')

In [95]:
ix = pd.Index([datetime.datetime(2017, 10, 1), datetime.datetime(2017, 10, 1)])
df.set_index(['grp', 'dt'])

date  flag
grp dt                                  
a   2017-10-01 2017-10-01 05:00:00     1
    2017-10-02 2017-10-02 05:00:00     1
    2017-10-02 2017-10-02 13:00:00     0
    2017-10-02 2017-10-02 14:00:00     1

In [114]:
d = d1 + datetime.timedelta(days=1)

In [116]:
datetime.datetime.combine(d, datetime.time())

datetime.datetime(2017, 10, 2, 0, 0)